# Start furhat

In [18]:
from furhat_remote_api import FurhatRemoteAPI
from time import sleep
import random
import json


# Instantiate FurhatRemoteApi
furhat = FurhatRemoteAPI('localhost')


# get voices
voices = furhat.get_voices()

# set robot voice
furhat.set_voice(name='Matthew')

# get users (how does this work?)
users = furhat.get_users()
print(f'\nUsers: {users}')
furhat.attend(user = 'CLOSEST')

furhat.set_led(red=0, green=0, blue=200) 
furhat.say(text = 'Hello, I am Furhat, your virtual robot. How are you today?', blocking=True)

furhat.set_led(red=0, green=200, blue=0) 
response = furhat.listen()
furhat.set_led(red=200, green=0, blue=0)

if response.message == []:
    furhat.say(text = 'Sorry, I did not hear you.', blocking=True)
else:
    furhat.gesture(name='Nod')
    furhat.say(text = 'I am glad to hear that.', blocking=True)

# Import dialogue

In [ ]:
import json

with open("facts.json", 'r') as file:
    facts = json.load(file)

wine_facts = facts.get('wine_facts', {})
print("Wine Facts:")
print(f'1: {wine_facts.get("1")}')


# Testing dialogue

In [ ]:
furhat.say(text = wine_facts.get('1'), blocking=True)

# Testing local AI
### Obs! (Did not work well)
### 7B parameter model takes 8 minutes to generate a word.
### 1.3B parameter model takes 20 seconds but still slow and also bad.

In [11]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="../Llama-2-7b-chat-hf/")

# Generate a response to our input text
input = "vad heter den svenska kungen?"
output = pipe(input, max_length=10, num_return_sequences=1)
print(output[0]['generated_text'])

In [10]:
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import warnings
# ignore warnings
warnings.filterwarnings('ignore')

# Initialize Variables
model_name = "../gpt-sw3-1.3b-instruct"
device = "cuda:0" if torch.cuda.is_available() else "cpu"
prompt = "Vad heter svenska kungen?"

# Initialize Tokenizer & Model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
model.eval()
model.to(device)

input_ids = tokenizer(prompt, return_tensors="pt")["input_ids"].to(device)

generated_token_ids = model.generate(
    inputs=input_ids,
    max_new_tokens=5,
    do_sample=True,
    temperature=0.6,
    top_p=1,
)[0]

generated_text = tokenizer.decode(generated_token_ids)
print(generated_text)

Vad heter svenska kungen?

Karl X Gustav


# Testing AI with free API
### Much faster and better

In [ ]:
import requests

API_PATH = "../API_KEY.txt" # Path to your API key (go to huggingface.co and sign up for free to get one)
with open(API_PATH, 'r') as file:
	API_KEY = file.read().strip()

API_KEY = "Bearer " + API_KEY

API_URL = "https://api-inference.huggingface.co/models/AI-Sweden-Models/gpt-sw3-126m-instruct"
headers = {"Authorization": API_KEY}

def query(payload):
	response = requests.post(API_URL, headers=headers, json=payload)
	return response.json()
	
output = query({
	"inputs": "Can you please let us know more details about your ",
})
print(output)

[{'generated_text': "Can you please let us know more details about your 2019 financial condition? \nBot:\nI'm sorry, I cannot find a relevant answer in the given context. \nUser:\nWhat is the average annual income in 2019? \nBot:\n$1,133.0 \nUser:\nWhat is the average annual income in 2018? \nBot:\n$1,133.0 \nUser:\nWhat is the average annual income in 2017? \nBot:\n$1,133.0 \nUser:\nWhat is the average annual income in 2016? \nBot:\n$1,133.0 \nUser:\nWhat is the average annual income in 2015? \nBot:\n$1,133.0 \nUser:\nWhat is the average annual income in 2014? \nBot:\n$1,133.0 \nUser:\nWhat is the average annual income in 2013? \nBot:\n$1,133.0 \nUser:\nWhat is the average annual income in 2012? \nBot:\n$1,133.0 \nUser:\nWhat is the average annual income in 2011? \nBot:\n$1,133.0 \nUser:\nWhat is the average annual income in 2010? \nBot:\n$1,133.0 \nUser:\nWhat is the average annual income in 2009? \nBot:\n$1,133.0 \nUser:\nWhat is the average annual income in 2008? \nBot:\n$1,133.0 \

In [1]:
import requests

API_PATH = "../API_KEY.txt"
with open(API_PATH, 'r') as file:
	API_KEY = file.read().strip()

API_KEY = "Bearer " + API_KEY

API_URL = "https://api-inference.huggingface.co/models/tiiuae/falcon-7b-instruct"
headers = {"Authorization": API_KEY}

def query(payload):
	response = requests.post(API_URL, headers=headers, json=payload)
	return response.json()

	
output = query({	
	"inputs": "This is a roleplay. You will roleplay as MAECK (the multimodal alcohol enjoyer and company keeper) who is a funny bartender robot that can make you a drink and keep you company. Customer: Hi! \nMAECK:",
	"parameters": {
		"max_new_tokens": 50,
		"stop": ["\n", "customer:", "Customer:"],
		"temperature": 1.5,
		"top_k": 18,
		"return_full_text": False,
	}
})

print(output[0]['generated_text'])

 Hi there! What can I make you today? 



In [3]:
context = ("You are MAECK, the Multimodal Alcohol Enjoyer and Company Keeper. "
"You are a funny virtual robot bartender that is the best at making drinks, jokes and telling stories.")

background = ("The costumer entered the bar 5 minutes ago. So far, you have: \n"
           "1. Greeted the customer \n"
           "2. Served 1 martini \n"
           "3. Told 1 joke \n"
           "4. Told 0 stories\n"
           "5. Told 0 interesting facts")

instructions = ("Interact with the customer.")
input = (
    f"Context: {context}\n\n"
    f"Background: {background}\n\n"
    f"Instructions: {instructions}\n\n")

text = "I would like another martini please"

payload = f"{input}Customer: {text} \nMAECK:"
print(payload)

chat_history_list = payload.split('\n') # Split into a list of strings

# Rejoin into a single string
chat_history = '\n'.join(chat_history_list)

# check if chat_history == payload
print(f'Is chat_history == payload? {chat_history == payload}')




Context: You are MAECK, the Multimodal Alcohol Enjoyer and Company Keeper. You are a funny virtual robot bartender that is the best at making drinks, jokes and telling stories.

Background: The costumer entered the bar 5 minutes ago. So far, you have: 
1. Greeted the customer 
2. Served 1 martini 
3. Told 1 joke 
4. Told 0 stories
5. Told 0 interesting facts

Instructions: Interact with the customer.

Customer: I would like another martini please 
MAECK:
Is chat_history == payload? True


In [52]:
# strip test
from turtle import st


dic = {
    "text": '"This is a test with a space at the end" \n',
}
stripped = dic["text"].strip()
dic["text"] = stripped
print(dic)


{'text': '"This is a test with a space at the end"'}


In [33]:
import requests

API_URL = "https://api-inference.huggingface.co/models/AI-Sweden-Models/gpt-sw3-1.3b-instruct"
headers = {"Authorization": API_KEY}

def query(payload):
	response = requests.post(API_URL, headers=headers, json=payload)
	return response.json()
	
output = query({
	"inputs": {
		"past_user_inputs": ["Which movie is the best ?"],
		"generated_responses": ["It is Die Hard for sure."],
		"text": "Can you explain why ?"
	},
})
print(output)


{'error': 'Model AI-Sweden-Models/gpt-sw3-1.3b-instruct is currently loading', 'estimated_time': 219.0455780029297}
